# Исследование надёжности заёмщиков

**Цель исследования** - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Для этого проверим гипотезы:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

В качестве входных данных используется статистика о платёжеспособности клиентов банка.

**Ход исследования**

 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Согласно документации к данным:**

`children` — количество детей в семье  
`days_employed` — общий трудовой стаж в днях  
`dob_years` — возраст клиента в годах  
`education` — уровень образования клиента  
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose` — цель получения кредита  

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**По первым строкам сразу видны проблемы с данными, а именно:**

* Общий трудовой стаж представлен в виде вещественного числа, а так же имеются отрицательные значения.
* Уровень образования записан в разном регистре
* Цель кредита записана синонимичными выражениями, такими как "покупка жилья", "покупка жилья для семьи", что по сути является просто приобретением жилья.

In [5]:
data.nunique()

children                8
days_employed       19351
dob_years              58
education              15
education_id            5
family_status           5
family_status_id        5
gender                  3
income_type             8
debt                    2
total_income        19351
purpose                38
dtype: int64

In [6]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Действительно, синонимичных выражений в целях кредита намного больше.

Проверим описательную статистику:

In [7]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


### Вывод

К озвученым ранее проблемам, а именно: отрицательных значениях где их не должно быть, разного регистра и синонемичных выражений, можно добавить слишком большое количество отработанных дней, в разы больше чем человеческая жизнь, а так же количество гендеров больше 2.

## Предобработка данных

**Что бы предобработать данные, выполним следующие действия:**

- Обработаем пропуски;
- Заменим типы данных;
- Обработаем дубликаты;
- Выполним лемматизацию;
- Выполним категоризацию.

### Обработка пропусков

In [8]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски имеются в общем трудовом стаже и ежемесячном доходе.

In [9]:
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Можно сделать вывод что это либо техническая причина, либо клиент сам не указал свой стаж и доход. В любом случае пропуски можно заполнить медианой дохода, на основе типа занятости.

Но для начала избавимся от артефактов: отрицательных значений и слишком большого стажа.

In [10]:
data['days_employed'] = abs(data['days_employed'])

In [11]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Как можно видеть даже средний стаж выбивается за рамки, за основу возьмем приблизительный максимум возможного стажа: работа началась в 18 лет и закончилась в 65, значит `237 рабочих дней * 47 лет = 11139 дней`

In [12]:
MAX_DAYS_EMPLOYED = 11139

In [13]:
data[data['days_employed'] > MAX_DAYS_EMPLOYED]['days_employed'].count()

3606

Получается 3606 выбивающихся из нормы значений.

In [14]:
data[data['days_employed'] > MAX_DAYS_EMPLOYED]['income_type'].value_counts()

пенсионер      3443
сотрудник        98
компаньон        37
госслужащий      26
безработный       2
Name: income_type, dtype: int64

Больше всего из них пенсионеров. Узнаем сколько всего пенсионеров в датасете:

In [15]:
data[(data['income_type'] == 'пенсионер')]['children'].count()

3856

И сколько у них стаж:

In [16]:
data[(data['income_type'] == 'пенсионер')]['days_employed'].describe()

count      3443.000000
mean     365003.491245
std       21069.606065
min      328728.720605
25%      346649.346146
50%      365213.306266
75%      383231.396871
max      401755.400475
Name: days_employed, dtype: float64

Получается весь стаж пенсионеров некорректен, для того чтобы не терять данный тип занятости, заполним его средним.

In [17]:
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] = \
    data[data['days_employed'] < MAX_DAYS_EMPLOYED]['days_employed'].mean()

Заполним остальной пропущенный стаж по оставшимся категориям занятости выбивающимся от нормы, для этого получим их среднее, а для непропущенных приведем к максимальному числу 11139:

In [18]:
days_employed_deviation = (data 
                            [data['days_employed'] > MAX_DAYS_EMPLOYED] 
                            ['income_type']
                           .value_counts())

In [19]:
days_employed_deviation_dict = {}
for key in list(days_employed_deviation.index):
    days_employed_deviation_dict[key] = (data[(data['days_employed'].notna()) 
                                             & (data['days_employed'] <= MAX_DAYS_EMPLOYED) 
                                             & (data['income_type'] == key)] 
                                            ['days_employed'] 
                                            .mean())                                            

И заполним:

In [20]:
# Заполнение стажа
for key, value in days_employed_deviation_dict.items():
    data.loc[((data['income_type'] == key) 
             & (data['days_employed'].isna()), 
              'days_employed')] = value

In [21]:
for key, value in days_employed_deviation_dict.items():
    data.loc[((data['income_type'] == key) 
             & (data['days_employed'] > MAX_DAYS_EMPLOYED), 
              'days_employed')] = MAX_DAYS_EMPLOYED

In [22]:
data.isna().sum()

children               0
days_employed          1
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверим сколько всего безработных:

In [23]:
data[(data['income_type'] == 'безработный')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,11139.0,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,11139.0,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Так как их всего два и на проверку гипотез эти данные не скажутся, можно их удалить:

In [24]:
data.drop(data[data['income_type'] == 'безработный'].index, inplace=True)

После удаления, проверим, остались ли пропуски в days_employed:

In [25]:
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости


Действительно, имеется один пропуск, узнаем сколько всего предпринимателей в датасете:

In [26]:
data[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


Так как у нас всего два значения из предпринимателей, поступим так же как и с безработными.

In [27]:
data.drop(data[data['income_type'] == 'предприниматель'].index, inplace=True)

Разберемся с пропусками по доходу, для этого получим медиану дохода по типу занятости:

In [28]:
# Получение медианы по доходу
median_total_income = (data[data['total_income'].notna()] 
                        .groupby('income_type')['total_income'] 
                        .median() 
                        .to_dict()) 

И заполним:

In [29]:
# Заполнение дохода
for key, value in median_total_income.items():
    data.loc[((data['total_income'].isna()) 
              & (data['income_type'] == key), 
              'total_income')] = value

In [30]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Разберемся с гендером:

In [31]:
data.gender.value_counts()

F      14234
M       7286
XNA        1
Name: gender, dtype: int64

Отклонение одно, поэтому просто удалим его:

In [32]:
data.drop(data[data['gender'] == 'XNA'].index, inplace=True)

Осталось разобраться с отрицательным значением в количестве детей:

In [33]:
data[data['children'] < 0]['children'].count()

47

Целых 47 человек у которых -1 ребенок. Скорее всего это человеческая ошибка и минус здесь лишний.

In [34]:
data['children'] = abs(data['children'])

In [35]:
data.children.describe()

count    21520.000000
mean         0.543355
std          1.380013
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

#### Вывод

Обработка пропусков выявила следующие артефакты:

* Отрицательные значения в стаже
* Имеется стаж превышающий человеческую жизнь
* Весь стаж пенсионеров некорректен
* Так же была одна ошибка в гендере
* Количество детей не может быть отрицательным

Из этого следует:

* Отрицательные значения скорее всего человеческая ошибка при заполнении
* Гендерная ошибка под вопросом, так как XNA похож на NA, а это возможно ошибка выгрузки данных
* Стаж превышающий человеческую жизнь в 89% случаев был выявлен у пенсионеров, а так как их стаж был полностью некорректен такое большое число вряд ли было введено человеком, имеет место быть техническая ошибка.

### Замена типа данных

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21520 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21520 non-null  int64  
 1   days_employed     21520 non-null  float64
 2   dob_years         21520 non-null  int64  
 3   education         21520 non-null  object 
 4   education_id      21520 non-null  int64  
 5   family_status     21520 non-null  object 
 6   family_status_id  21520 non-null  int64  
 7   gender            21520 non-null  object 
 8   income_type       21520 non-null  object 
 9   debt              21520 non-null  int64  
 10  total_income      21520 non-null  float64
 11  purpose           21520 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Можно отметить, что столбец days_employed имеет неправильный тип данных, должен быть целочисленный, так же для удобства и экономии памяти переведем:
- children в uint8
- days_employed в uint16
- dob_years в uint8
- education_id в uint8
- family_status_id в uint8
- debt в uint8
- total_income в uint32

In [37]:
data['children'] = data['children'].astype('uint8')

In [38]:
data['days_employed'] = data['days_employed'].astype('uint16')

In [39]:
data['dob_years'] = data['dob_years'].astype('uint8')

In [40]:
data['education_id'] = data['education_id'].astype('uint8')

In [41]:
data['family_status_id'] = data['family_status_id'].astype('uint8')

In [42]:
data['debt'] = data['debt'].astype('uint8')

In [43]:
data['total_income'] = data['total_income'].astype('uint32')

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21520 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21520 non-null  uint8 
 1   days_employed     21520 non-null  uint16
 2   dob_years         21520 non-null  uint8 
 3   education         21520 non-null  object
 4   education_id      21520 non-null  uint8 
 5   family_status     21520 non-null  object
 6   family_status_id  21520 non-null  uint8 
 7   gender            21520 non-null  object
 8   income_type       21520 non-null  object
 9   debt              21520 non-null  uint8 
 10  total_income      21520 non-null  uint32
 11  purpose           21520 non-null  object
dtypes: object(5), uint16(1), uint32(1), uint8(5)
memory usage: 1.2+ MB


#### Вывод

В столбце days_employed вещественный тип данных был переведен в целочисленный, так как рабочий стаж в днях не может быть дробным. Причина появления кроется в наличии NaN при выгрузке данных. Так же после перевода других столбцов размер датафрейма сократился почти вдвое.

### Обработка дубликатов

In [45]:
data.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [46]:
data['education'] = data['education'].str.lower()

In [47]:
data.duplicated().sum()

71

In [48]:
data.drop_duplicates(inplace=True)

In [49]:
data = data.reset_index(drop=True)

#### Вывод
 
Столбец education имел одинаковые значения в разном регистре, после приведения к нижнему регистру и применению метода drop_duplicates, датасет больше не имеет дубликатов. Причина разного регистра, скорее всего, в том что, образование вводится вручную, а наличие дубликатов в том что один человек может подавать несколько раз заявки на кредит.

### Лемматизация

Проведем лемматизацию столбца с целями кредита.

In [50]:
def create_lemma_purpose(row, instance):
    """
    row - строка датафрейма
    instance - экземпляр класса для лемматизации
    
    """
    lemmas = instance.lemmatize(row['purpose'])

    for lemma in lemmas:
        if lemma.startswith('авто'):
            return 'автомобиль'
        elif lemma.startswith('свадьб'):
            return 'свадьба'
        elif lemma.startswith('недвижимост') or lemma.startswith('жиль'):
            return 'недвижимость'
        elif lemma.startswith('образован'):
            return 'образование'
    return np.nan

In [51]:
m = Mystem()

In [52]:
data['lemmatize_purpose'] = data.apply(create_lemma_purpose, instance=m, axis=1)

In [53]:
data[['purpose', 'lemmatize_purpose']].head(10)

,purpose,lemmatize_purpose
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
5,покупка жилья,недвижимость
6,операции с жильем,недвижимость
7,образование,образование
8,на проведение свадьбы,свадьба
9,покупка жилья для семьи,недвижимость


In [54]:
data.lemmatize_purpose.isna().sum()

0

#### Вывод

Была проведена лемматизация целей кредита с помощью библиотеки pymystem3. Теперь осуществлять поиск по целям кредита стало проще.

### Категоризация данных

Для проверки гипотез объединим данные в категории:

1. Наличие/отсутствие детей
2. Наличие/отсутствие брака
3. Уровень дохода

In [55]:
# Категоризация по наличию детей
data['have_children'] = data['children'].apply(lambda x: x > 0)

Выделим следующие типы отношений:

In [56]:
family_status = data[['family_status_id', 'family_status']]

In [57]:
family_status.drop_duplicates().reset_index(drop=True)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Имеется 5 типов которые можно классифицировать: имеется брак `family_status_id = 0, 1`/брак отсутствует `family_status_id = 2, 3, 4`, все это будет выведено в столбец `is_married`.

In [58]:
# Категоризация по наличию брака
data['is_married'] = data['family_status_id'].apply(lambda x: x == 0 or x == 1)

За уровень дохода примем 25, 50, 75 процентили. Выделим 4 подкатегории: меньше 25 процентиля - очень низкий, между 25 и 50 - средний, от 50 до 75 - высокий, выше 75 - очень высокий.

In [59]:
# Категоризация по уровню дохода
def get_string_income(income, low, middle, high):
    """
    income - уровень дохода
    low - уровень дохода соответствующий низкому
    middle - уровень дохода соответствующий среднему
    high - уровень дохода соответствующий высокому
    
    """
    
    if income < low:
        return 'очень низкий'
    elif low <= income <= middle:
        return 'средний'
    elif middle < income <= high:
        return 'высокий'
    else:
        return 'очень высокий'

In [60]:
low = data['total_income'].quantile(0.25)
middle = data['total_income'].quantile(0.5)
high = data['total_income'].quantile(0.75)

In [61]:
data['income'] = (data['total_income'] 
                .apply(get_string_income, low=low, middle=middle, high=high))

#### Вывод

Были выделены три категории: наличие брака, наличие детей, уровень дохода, все эти категории помогут ответить на выдвинутые гипотезы. 

## Проверка гипотез

***Проверим гипотезу: есть ли зависимость между наличием детей и возвратом кредита в срок?***

In [62]:
data.groupby('have_children', as_index=False).agg({'debt': 'mean'})

,have_children,debt
0,False,0.075460
1,True,0.091959


Наличие детей увеличивает шанс невозврата кредита. Проверим, увеличивает ли этот процент количество детей.

In [63]:
data.groupby('children', as_index=False).agg({'debt': 'mean'})

,children,debt
0,0,0.075460
1,1,0.091471
2,2,0.094542
3,3,0.081818
4,4,0.097561
5,5,0.000000
6,20,0.105263


Цифры плюс-минус одинаковые, если не считать случаев когда количество детей равно 5.

**Вывод**

Гипотеза подтвердилась, наличие детей сказывается на шанс невозврата кредита, количество детей практически не сказывается на этом показателе.

***Проверим гипотезу: есть ли зависимость между семейным положением и возвратом кредита в срок?***

In [64]:
data.groupby('is_married', as_index=False).agg({'debt': 'mean'})

,is_married,debt
0,False,0.085012
1,True,0.079951


**Вывод**

Зависимость действительно есть, люди несостоящие в браке реже возвращают кредит.

***Проверим гипотезу: есть ли зависимость между уровнем дохода и возвратом кредита в срок?***

In [65]:
data.groupby('income', as_index=False).agg({'debt': 'mean'}).sort_values('debt')

,income,debt
1,очень высокий,0.071429
2,очень низкий,0.079448
0,высокий,0.085415
3,средний,0.088139


**Вывод**

Самыми платежеспособными оказались те у кого доход или очень низкий, или очень высокий, у кого же доход ближе к среднему хуже платят по кредиту.

***Проверим гипотезу: как разные цели кредита влияют на его возврат в срок?***

In [66]:
data.groupby('lemmatize_purpose', as_index=False).agg({'debt': 'mean'}).sort_values('debt')

,lemmatize_purpose,debt
1,недвижимость,0.072268
3,свадьба,0.080069
2,образование,0.092200
0,автомобиль,0.093590


**Вывод**

Самой выгодной целью в плане кредитования оказалась недвижимость, затем свадьба, образование и завершает все покупка автомобиля.

## Общий вывод

* Первая гипотеза подтвердилась, наличие детей действительно сказывается на шанс невозврата кредита, а вот количество практически не сказывается на этом показателе.
* Вторая гипотеза тоже подтвердилась зависимость действительно есть, люди несостоящие в браке реже возвращают кредит.
* Самыми платежеспособными оказались те у кого доход или очень низкий, или очень высокий, у кого же доход ближе к среднему хуже платят по кредиту.
* Самой выгодной целью в плане кредитования оказалась недвижимость, затем свадьба, образование и завершает все покупка автомобиля.


Узнаем какой тип человека реже всего отдает кредит, для этого соберем данные в сводную таблицу, так как наличие детей напрямую сказывается на невозврат кредита, включим только эту категорию.

In [67]:
conclusion = (data.query("have_children == True"). 
                    pivot_table(index='lemmatize_purpose', 
                                 columns=['income', 'is_married'], 
                                 values='debt', 
                                 aggfunc='mean', 
                                 fill_value=0))

In [68]:
conclusion

income              высокий           очень высокий           очень низкий  \
is_married            False      True         False      True        False   
lemmatize_purpose                                                            
автомобиль         0.250000  0.110749      0.094340  0.090909     0.020408   
недвижимость       0.102410  0.076923      0.095541  0.072202     0.097222   
образование        0.038462  0.109635      0.060000  0.097378     0.064516   
свадьба            0.000000  0.115789      0.000000  0.029268     0.000000   

income                        средний            
is_married             True     False      True  
lemmatize_purpose                                
автомобиль         0.131387  0.117647  0.101307  
недвижимость       0.078103  0.081761  0.089514  
образование        0.092527  0.152542  0.127090  
свадьба            0.138462  0.000000  0.078431

Теперь оставим только тот тип заемщика который хуже всего отдает кредит.

In [69]:
debt = (conclusion
        .unstack()
        .to_frame() 
        .rename(columns={0: 'debt'}) 
        .reset_index() 
        .query("debt > 0") 
        .sort_values('debt'))

In [70]:
debt.tail(1)

,income,is_married,lemmatize_purpose,debt
0,высокий,False,автомобиль,0.25


Как можно видеть, кредит выданный человеку с высоким уровнем дохода, не состоящему в браке, с ребенком и под покупку автомобиля невозвращается в 25% случаев.

Так же хотелось бы отметить что для данного исследования стаж, образование и гендер не представляли интереса, и именно в этих данных закрались ошибки. Регистр в образовании легко поправить если пользователь будет его вводить не вручную, а путем выбора из предложенных вариантов, гендер, скорее всего, просто ошибка при выгрузке данных. Что касается стажа, то тут либо человеческая, либо техническая ошибка и последнее скорее всего, так как кроме отрицательных значений, предобработка выявила слишком большие цифры, которые вряд ли ввел бы человек.